# Read BGS Coadds from Feb 26 2020

In [1]:
import numpy as np

from astropy.io import fits
from astropy.table import Table, vstack
import matplotlib.pyplot as plt

import desitarget
from desitarget.targetmask import desi_mask, bgs_mask
from desitarget.cmx import cmx_targetmask
from desispec.io import read_spectra

from glob import glob

from scipy.ndimage import median_filter

## Mini-SV2 BGS Tile 70502

In [2]:
prefix = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225'
coadds = sorted(glob('{}/*coadd*.fits'.format(prefix)))
coadds

['/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-0-70502-20200225.fits',
 '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-3-70502-20200225.fits',
 '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-6-70502-20200225.fits',
 '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-7-70502-20200225.fits',
 '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-9-70502-20200225.fits']

### Spin through Coadd Files, Load Spectra

In [3]:
fibermap = None
for coadd in coadds:
    hdus = fits.open(coadd)
    spectra = read_spectra(coadd)
    if fibermap is None:
        fibermap = spectra.fibermap
    else:
        fibermap = vstack([fibermap, spectra.fibermap])

### Apply Bitmasks

In [4]:
bgs_bright = fibermap['CMX_TARGET'] & cmx_targetmask.cmx_mask.mask('MINI_SV_BGS_BRIGHT') != 0
sv0_bgs    = fibermap['CMX_TARGET'] & cmx_targetmask.cmx_mask.mask('SV0_BGS') != 0
is_bgs = bgs_bright | sv0_bgs

In [5]:
np.sum(is_bgs)

1484

In [6]:
fibermap[is_bgs]

TARGETID,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,PMRA_IVAR,PMDEC_IVAR,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,NUMTARGET,PRIORITY,SUBPRIORITY,OBSCONDITIONS,NUMOBS_MORE,RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,TARGET_RA_IVAR,TARGET_DEC_IVAR,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,WISEMASK_W1,WISEMASK_W2,MASKBITS,FRACDEV,FRACDEV_IVAR,SHAPEDEV_R,SHAPEDEV_E1,SHAPEDEV_E2,SHAPEDEV_R_IVAR,SHAPEDEV_E1_IVAR,SHAPEDEV_E2_IVAR,SHAPEEXP_R,SHAPEEXP_E1,SHAPEEXP_E2,SHAPEEXP_R_IVAR,SHAPEEXP_E1_IVAR,SHAPEEXP_E2_IVAR,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PHOTSYS,CMX_TARGET,PRIORITY_INIT,NUMOBS_INIT,HPXPIXEL,BLOBDIST,FIBERFLUX_IVAR_G,FIBERFLUX_IVAR_R,FIBERFLUX_IVAR_Z,DESI_TARGET,BGS_TARGET,MWS_TARGET,NUM_ITER,FIBER_X,FIBER_Y,MEAN_DELTA_X,MEAN_DELTA_Y,FIBER_RA,FIBER_DEC,RMS_DELTA_X,RMS_DELTA_Y,FIRST_FIBER,LAST_FIBER,NUM_FIBER
int64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,int64,uint8,bytes3,float32,float32,int16,int32,float64,int32,int32,int16,int32,bytes8,int32,bytes4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,bytes2,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,bool,float32,float32,bytes1,int64,int64,int64,int64,float32,float32,float32,float32,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64
35185712445788791,0,477,477,385,512,179.7394266438731,-1.9417105015239313,0.0,0.0,0.0,0.0,0.0,5400.0,256,1,TGT,65.62806,-363.29373,0,1000,0.9024699105607836,65535,99,8000,319566,1796m020,4727,EXP,245955760000.0,497515660000.0,0.029821675,6.701717,11.84153,17.539038,550.02655,209.74538,32.81296,0.9155061,0.94226795,0.96728486,0.0,0.0,0.0,0.0041720816,0.006273054,0.0067185066,0.99747866,0.99632025,0.99579763,6,6,4,2718.5935,931.11285,154.94714,1736.8547,590.7989,86.91487,10.385784,3.846617,-23.612312,171.84386,2.074136,0.35448593,0.0005257642,7.381861e-06,0.9949589,0.9969011,0.99933827,0.9997501,0,0,0,1.654141,2.9227672,4.329046,1.654141,2.9227672,4.329046,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.47261,-0.28731322,0.060808267,18385.857,51383.84,27894.914,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,True,0.0,0.0,S,256,2100,1,25596,0.0,0.0,0.0,0.0,0,0,0,3,68.255,-364.545,-2.687666666666667,1.6769999999999998,0.0,0.0,2.6876822728886687,1.6770543620686043,385,385,1
35185712449979800,0,522,522,478,0,179.8594279099647,-2.0776414098954636,0.0,0.0,0.0,0.0,0.0,5400.0,256,1,TGT,35.616833,-399.96597,0,1000,0.44963994293922915,65535,99,8000,319567,1798m020,1432,EXP,565765140000.0,487263570000.0,0.028192783,3.8041756,6.9930053,9.800576,1132.5394,388.475

In [7]:
columns = ['TARGETID', 'TARGET_RA', 'TARGET_DEC']

In [8]:
idx = np.random.choice(np.sum(is_bgs), 4)
fibermap[is_bgs][columns][idx]

TARGETID,TARGET_RA,TARGET_DEC
int64,float64,float64
35185730573569617,180.14947636101462,-1.1826312742940237
35185730573568587,180.1046308933762,-1.2410810782483876
35185718498166253,180.48629116306978,-1.80726313694911
35185718502362542,180.69245682124182,-1.6828350028349215


In [9]:
help(spectra)

Help on Spectra in module desispec.spectra object:

class Spectra(builtins.object)
 |  Represents a grouping of spectra.
 |  
 |  This class contains an "extended" fibermap that has information about
 |  the night and exposure of each spectrum.  For each band, this class has 
 |  the wavelength grid, flux, ivar, mask, and resolution arrays.
 |  
 |  Parameters
 |  ----------
 |  bands : :class:`list`
 |      List of strings used to identify the bands.
 |  wave : :class:`dict`
 |      Dictionary of arrays specifying the wavelength grid.
 |  flux : :class:`dict`
 |      Dictionary of arrays specifying the flux for each spectrum.
 |  ivar : :class:`dict`
 |      Dictionary of arrays specifying the inverse variance.
 |  mask : :class:`dict`, optional
 |      Dictionary of arrays specifying the bitmask.
 |  resolution_data : :class:`dict`, optional
 |      Dictionary of arrays specifying the block diagonal resolution matrix.
 |      The object for each band must be in one of the formats sup